In [1]:
"""
Data exploration notebook for viewing missing values in suicide study dataset.
"""
import sys
from pathlib import Path
import pandas as pd
import os

# Add project root to Python path
PROJECT_ROOT = Path.cwd().parent

if "notebooks" in Path.cwd().parts:
    os.chdir(PROJECT_ROOT)
    
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

# Import custom utility functions
from src.config.utils import read_csv, read_excel, write_excel
from src.config.config import DATA_DIR, RESULTS_DIR


In [17]:
def analyze_dataframe_columns(dataframe):
    """
    Analyzes DataFrame columns by calculating missing data statistics and unique value counts.

    Args:
        dataframe (pd.DataFrame): Input DataFrame for analysis.

    Returns:
        pd.DataFrame: Analysis results with columns:
            - column_name: Name of each column
            - missing_values_total: Count of missing values
            - missing_values_percent: Percentage of missing values
            - unique_values_count: Count of unique values
            - unique_value_counts: String of unique values and their counts
    """
    results = []

    for col in dataframe.columns:
        missing_total = dataframe[col].isnull().sum()
        missing_percent = 100 * missing_total / len(dataframe)

        unique_count = dataframe[col].nunique()
        value_counts = dataframe[col].value_counts().to_dict()
        value_counts_str = ", ".join([f"{k}: {v}" for k, v in value_counts.items()])

        results.append(
            [col, missing_total, missing_percent, unique_count, value_counts_str]
        )

    analysis_df = pd.DataFrame(
        results,
        columns=[
            "column_name",
            "missing_values_total",
            "missing_values_percent",
            "unique_values_count",
            "unique_value_counts",
        ],
    )

    # Sort the DataFrame by missing_values_total in descending order
    analysis_df = analysis_df.sort_values(by="missing_values_total", ascending=True)

    return analysis_df


def nan_exploration_in_rows(dataframe):
    """
    Analyzes NaN distribution across DataFrame rows.

    Args:
        dataframe (pd.DataFrame): Input DataFrame for analysis.

    Returns:
        pd.DataFrame: Analysis results with columns:
            - NaN_count: Number of NaN values in each row
            - Total: Count of rows with that NaN count
            - Percent: Percentage of rows with that NaN count
    """
    nan_counts = dataframe.isna().sum(axis=1).value_counts()
    full_index = list(range(0, len(dataframe.columns) + 1))
    nan_counts = nan_counts.reindex(full_index, fill_value=0)
    nan_counts = nan_counts.sort_index()
    nan_counts_percent = (nan_counts / len(dataframe)) * 100

    missing_data_rows = pd.concat(
        [
            pd.Series(full_index, name="NaN_count"),
            nan_counts.rename("Total"),
            nan_counts_percent.rename("Percent"),
        ],
        axis=1,
    )

    return missing_data_rows

# raw data

In [18]:
# Data import
csv_file_path = DATA_DIR / "raw" / "final_samobojstwa_2013_2022.csv"
df_raw_2013_2022 = read_csv(csv_file_path, low_memory=False)

excel_file_path = DATA_DIR / "raw" / "Samobojstwa_2023.xlsx"
df_raw_2023 = read_excel(excel_file_path)

In [19]:
df_raw_2013_2022.columns

Index(['ID samobójcy', 'Data raportu [RRRRMM]', 'Przedział wiekowy', 'Płeć',
       'Stan cywilny', 'Wykształcenie', 'Informacje o pracy i nauce',
       'Źródło utrzymania', 'Czy samobójstwo zakończyło się zgonem',
       'Miejsce zamachu', 'Sposób popełnienia', 'Powód zamachu',
       'Powód zamachu 2', 'Powód zamachu 3', 'Powód zamachu 4',
       'Stan świadomości', 'Informacje o używaniu substancji',
       'Informacje dotyczące leczenia z powodu alkoholizmu/narkomanii'],
      dtype='object')

In [20]:
df_raw_2013_2022.shape

(113196, 18)

In [21]:
df_raw_2013_2022["ID samobójcy"].nunique()

113196

In [22]:
df_raw_2023.columns

Index(['ID samobójcy', 'Data rejestracji', 'Przedział wiekowy', 'Płeć',
       'Stan cywilny', 'Wykształcenie', 'Informacje o pracy i nauce',
       'Źródło utrzymania', 'Czy samobójstwo zakończyło się zgonem',
       'Miejsce zamachu', 'Klasa miejscowości', 'Sposób popełnienia',
       'Powód zamachu *', 'Powód zamachu 2', 'Powód zamachu 3',
       'Powód zamachu 4', 'Stan świadomości *',
       'Informacje dotyczące stanu zdrowia *',
       'Informacje dotyczące leczenia z powodu alkoholizmu/narkomanii',
       'W ciągu ostatniego miesiąca sprawca zdarzenia miał przynajmniej jeden raz kontakt z *'],
      dtype='object')

In [23]:
df_raw_2023.shape

(15136, 20)

In [24]:
df_raw_2023["ID samobójcy"].nunique()

15134

In [25]:
df_raw_2023[df_raw_2023["ID samobójcy"].duplicated(keep=False)]

,ID samobójcy,Data rejestracji,Przedział wiekowy,Płeć,Stan cywilny,Wykształcenie,Informacje o pracy i nauce,Źródło utrzymania,Czy samobójstwo zakończyło się zgonem,Miejsce zamachu,Klasa miejscowości,Sposób popełnienia,Powód zamachu *,Powód zamachu 2,Powód zamachu 3,Powód zamachu 4,Stan świadomości *,Informacje dotyczące stanu zdrowia *,Informacje dotyczące leczenia z powodu alkoholizmu/narkomanii,W ciągu ostatniego miesiąca sprawca zdarzenia miał przynajmniej jeden raz kontakt z *
1399,NaN,NaT,30-34,Mężczyzna,Kawaler/panna,Brak danych/nieustalone,Bezrobotny,Brak danych/nieustalony,N,Mieszkanie/dom,Miasto o liczbie ludności 100 000 – 499 999,Zażycie innych leków,Nieustalony,NaN,NaN,NaN,Pod wpływem leków,Nadużywał(a) alkoholu,NaN,Brak możliwości ustalenia
5600,NaN,NaT,80-84,Kobieta,Wdowiec/wdowa,Podstawowe,Brak danych/nieustalono,Emerytura,N,Mieszkanie/dom,Miasto o liczbie ludności do 19 999,Zażycie środków nasennych/leków psychotropowych,Śmierć bliskiej osoby,NaN,NaN,NaN,Pod wpływem leków,Leczony(a) psychiatrycznie,NaN,Placówką leczniczą


In [26]:
df_raw_2023.rename(columns={"ID samobójcy": "ID"}, inplace=True)
df_raw_2013_2022.rename(columns={"ID samobójcy": "ID"}, inplace=True)

In [27]:
df_raw = pd.concat([df_raw_2013_2022[["ID"]], df_raw_2023[["ID"]]], ignore_index=True)
df_raw.shape

(128332, 1)

In [28]:
df_raw["ID"].nunique()

128330

In [29]:
df_raw[df_raw["ID"].duplicated(keep=False)]

,ID
114595,NaN
118796,NaN


# mapped_data

In [30]:
# Data import
csv_file_path = DATA_DIR / "processed" / "mapped_data.csv"
df_raw = read_csv(csv_file_path)

In [31]:
df_raw.shape

(128330, 25)

In [32]:
df_raw["ID"].nunique()

128330

In [33]:
# Split data and context
context_columns = [col for col in df_raw.columns if col.startswith("Context_")]
df_data = df_raw.drop(columns=context_columns, inplace=False)

In [34]:
# Data exploration
nan_columns_df = analyze_dataframe_columns(df_data)
nan_columns_df

,column_name,missing_values_total,missing_values_percent,unique_values_count,unique_value_counts
0,ID,0,0.000000,128330,"0: 1, 85549: 1, 85562: 1, 85561: 1, 85560: 1, ..."
1,Date,1,0.000779,29699,"2021-05-01: 1311, 2021-06-01: 1241, 2021-04-01..."
12,DateY,1,0.000779,11,"2023.0: 15133, 2022.0: 14517, 2021.0: 13793, 2..."
13,DateM,1,0.000779,12,"5.0: 11632, 6.0: 11550, 7.0: 11398, 8.0: 11100..."
3,Gender,7,0.005455,2,"M: 93811, F: 34512"
8,Fatal,71,0.055326,2,"0.0: 68489, 1.0: 59770"
10,Method,71,0.055326,11,"Hanging: 64233, Drugs: 21404, SelfHarm: 18453,..."
9,Place,107,0.083379,12,"House: 79745, Other: 13810, UtilitySpaces: 116..."
2,AgeGroup,1296,1.009896,16,"19_24: 13764, 30_34: 13377, 35_39: 13287, 25_2..."
4,Marital,16270,12.678251,7,"Single: 50741, Married: 40423, Divorced: 8782,..."


In [35]:
write_excel(
    file_path=RESULTS_DIR / "nan_exploration.xlsx",
    data=nan_columns_df,
    sheet_name="nan_columns",
    mode="w",
    index=False,
)

In [36]:
nan_rows_df = nan_exploration_in_rows(df_data)
nan_rows_df

,NaN_count,Total,Percent
0,0,5558,4.331022
1,1,21217,16.533157
2,2,24896,19.399984
3,3,22917,17.857866
4,4,24964,19.452973
5,5,21630,16.854983
6,6,6725,5.240396
7,7,345,0.268838
8,8,9,0.007013
9,9,8,0.006234


In [37]:
write_excel(
    file_path=RESULTS_DIR / "nan_exploration.xlsx",
    data=nan_rows_df,
    sheet_name="nan_rows",
    mode="a",
    index=False,
)